In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./input/train_data.csv
./input/test_data.csv
./input/sample_submission.csv


In [8]:
# Import additional required libraries

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

In [9]:
# Turn off any warnings

import warnings

warnings.filterwarnings('ignore')

In [10]:
# Load the datasets

train_data = pd.read_csv('/kaggle/input/credit-card-defaulter-hackathon/train_data.csv')
test_data = pd.read_csv('/kaggle/input/credit-card-defaulter-hackathon/test_data.csv')
sample_submission = pd.read_csv('/kaggle/input/credit-card-defaulter-hackathon/test_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/credit-card-defaulter-hackathon/train_data.csv'

In [ ]:
# Clean column names: strip leading/trailing spaces and replace unwanted characters

train_data.columns = train_data.columns.str.strip().str.replace(' ', '_')
test_data.columns = test_data.columns.str.strip().str.replace(' ', '_')

In [ ]:
# Examine the first 5 rows of the train data

train_data.head(5)

,ID,TARGET,Limit_balance,Sex,Education,Marriage,Age,Pay_1,Pay_2,Pay_3,...,Credit_tier,Bureau_score,Date_account_opened,Tenure_years,Credit_utilisation_ratio,Average_payment_delay,Total_payments,Total_bill_amount,Payment_consistency,Age_group
0,1,1,120000,2,2,2,26,-1,2,0,...,Basic,688,2017/07/22,6,0.14,0.50,5000,17077,753,Young
1,2,0,90000,2,2,2,34,0,0,0,...,Gold,482,2019/01/08,5,1.13,0.00,11018,101653,1570,Middle-aged
2,3,0,50000,1,2,1,57,-1,0,-1,...,Gold,560,2017/01/03,7,2.19,-0.33,59049,109339,13786,Senior
3,4,0,50000,1,1,2,37,0,0,0,...,Platinum,376,2021/02/08,3,4.76,0.00,7772,238114,714,Middle-aged
4,5,0,500000,1,1,2,29,0,0,0,...,Platinum,497,2017/01/08,7,5.45,0.00,180759,2724595,16803,Young


In [ ]:
# Examine the first 5 rows of the test data

test_data.head(5)

,ID,Limit_balance,Sex,Education,Marriage,Age,Pay_1,Pay_2,Pay_3,Pay_4,...,Credit_tier,Bureau_score,Date_account_opened,Tenure_years,Credit_utilisation_ratio,Average_payment_delay,Total_payments,Total_bill_amount,Payment_consistency,Age_group
0,33372,20000,2,2,1,24,2,2,-1,-1,...,Platinum,561,2018/07/10,5,0.39,0.33,689,7704,281,Young
1,33373,50000,2,2,1,37,0,0,0,0,...,Gold,570,2018/12/22,5,4.63,0.00,8388,231334,478,Middle-aged
2,33374,20000,1,3,2,35,-2,-2,-2,-2,...,Gold,364,2021/10/05,2,1.35,1.67,14129,26919,5238,Middle-aged
3,33375,70000,1,2,2,30,1,2,2,0,...,Basic,726,2020/06/29,3,4.84,1.17,10700,338685,1511,Middle-aged
4,33376,50000,2,3,3,23,1,2,0,0,...,Basic,332,2022/11/22,1,3.93,0.50,6200,196416,528,Young


In [ ]:
# Examine the first 5 rows of the submission data

sample_submission.head(5)

,ID,Limit_balance,Sex,Education,Marriage,Age,Pay_1,Pay_2,Pay_3,Pay_4,...,Credit_tier,Bureau_score,Date_account_opened,Tenure_years,Credit_utilisation_ratio,Average_payment_delay,Total_payments,Total_bill_amount,Payment_consistency,Age_group
0,33372,20000,2,2,1,24,2,2,-1,-1,...,Platinum,561,2018/07/10,5,0.39,0.33,689,7704,281,Young
1,33373,50000,2,2,1,37,0,0,0,0,...,Gold,570,2018/12/22,5,4.63,0.00,8388,231334,478,Middle-aged
2,33374,20000,1,3,2,35,-2,-2,-2,-2,...,Gold,364,2021/10/05,2,1.35,1.67,14129,26919,5238,Middle-aged
3,33375,70000,1,2,2,30,1,2,2,0,...,Basic,726,2020/06/29,3,4.84,1.17,10700,338685,1511,Middle-aged
4,33376,50000,2,3,3,23,1,2,0,0,...,Basic,332,2022/11/22,1,3.93,0.50,6200,196416,528,Young


In [ ]:
# Some columns which are expected to be numeric look to be string/categorical and need to be converted back to numerical

# List of numerical columns that might be read as strings

numerical_columns = ['Limit_balance', 'Age', 'Pay_1', 'Pay_2', 'Pay_3', 'Pay_4', 'Pay_5', 'Pay_6', 
                     'Bill_amt1', 'Bill_amt2', 'Bill_amt3', 'Bill_amt4', 'Bill_amt5', 'Bill_amt6', 
                     'Pay_amt1', 'Pay_amt2', 'Pay_amt3', 'Pay_amt4', 'Pay_amt5', 'Pay_amt6',
                     'Bureau_score', 'Tenure_years', 'Credit_utilisation_ratio', 
                     'Average_payment_delay', 'Total_payments', 'Total_bill_amount', 'Payment_consistency']

# Replace empty strings with np.nan and ensure numerical columns are converted to numeric types
for col in numerical_columns:
    train_data[col] = train_data[col].replace(' -   ', np.nan).astype(float)
    
for col in numerical_columns:
    test_data[col] = test_data[col].replace(' -   ', np.nan).astype(float)
    
# Ensure numerical columns are converted to numeric types and handle NaNs
for col in numerical_columns:
    if col in train_data.columns:
        train_data[col] = pd.to_numeric(train_data[col], errors='coerce').fillna(0)  # Fill NaNs with 0 or appropriate value
    if col in test_data.columns:
        test_data[col] = pd.to_numeric(test_data[col], errors='coerce').fillna(0)  # Fill NaNs with 0 or appropriate value

In [ ]:
# Examine the first 5 rows to see if the result is as expected
train_data.head(5)

,ID,TARGET,Limit_balance,Sex,Education,Marriage,Age,Pay_1,Pay_2,Pay_3,...,Credit_tier,Bureau_score,Date_account_opened,Tenure_years,Credit_utilisation_ratio,Average_payment_delay,Total_payments,Total_bill_amount,Payment_consistency,Age_group
0,1,1,120000.0,2,2,2,26.0,-1.0,2.0,0.0,...,Basic,688.0,2017/07/22,6.0,0.14,0.50,5000.0,17077.0,753.0,Young
1,2,0,90000.0,2,2,2,34.0,0.0,0.0,0.0,...,Gold,482.0,2019/01/08,5.0,1.13,0.00,11018.0,101653.0,1570.0,Middle-aged
2,3,0,50000.0,1,2,1,57.0,-1.0,0.0,-1.0,...,Gold,560.0,2017/01/03,7.0,2.19,-0.33,59049.0,109339.0,13786.0,Senior
3,4,0,50000.0,1,1,2,37.0,0.0,0.0,0.0,...,Platinum,376.0,2021/02/08,3.0,4.76,0.00,7772.0,238114.0,714.0,Middle-aged
4,5,0,500000.0,1,1,2,29.0,0.0,0.0,0.0,...,Platinum,497.0,2017/01/08,7.0,5.45,0.00,180759.0,2724595.0,16803.0,Young


In [ ]:
# Due to converting the categorical data to some numeric value the datasets need to be joined together
# Add a column to identify the rows in the combined dataset

train_data['is_train'] = 1
test_data['is_train'] = 0

In [ ]:
# Concatenate the training and test datasets

combined_data = pd.concat([train_data, test_data], ignore_index=True)

In [ ]:
# Convert categorical features to dummy/indicator variables

combined_data_dummies = pd.get_dummies(combined_data, columns=['City', 'State', 'Credit_tier', 'Age_group'], drop_first=True)

In [ ]:
# Split the combined dataset back into training and test sets

train_data_dummies = combined_data_dummies[combined_data_dummies['is_train'] == 1].drop(columns=['is_train'])
test_data_dummies = combined_data_dummies[combined_data_dummies['is_train'] == 0].drop(columns=['is_train'])

In [ ]:
# Split the data into features and target

X_train_full = train_data_dummies.drop(columns=['TARGET', 'Date_account_opened'])
y_train_full = train_data_dummies['TARGET']

In [ ]:
# The test data that will be used for the predictions for submission

X_test = test_data_dummies.drop(columns=['TARGET', 'Date_account_opened'])

In [ ]:
# Split the full training data into training and validation sets

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=2024)

In [ ]:
# Train a logistic regression model

model = LogisticRegression(max_iter=500, random_state=2024)
model.fit(X_train, y_train)

LogisticRegression(max_iter=500, random_state=2024)

In [ ]:
# View the model performance 
# Make predictions with the model on the validation set

y_pred_val = model.predict(X_val)

# Evaluate the model using F1-score on the validation set

f1 = f1_score(y_val, y_pred_val)
print(f'F1-Score on validation data: {f1}')
print('Classification Report on validation data:')
print(classification_report(y_val, y_pred_val))

F1-Score on validation data: 0.0067294751009421274
Classification Report on validation data:
              precision    recall  f1-score   support

         0.0       0.78      1.00      0.88      5199
         1.0       0.50      0.00      0.01      1476

    accuracy                           0.78      6675
   macro avg       0.64      0.50      0.44      6675
weighted avg       0.72      0.78      0.68      6675



In [ ]:
# Make predictions with the model
y_pred_test = model.predict(X_test)

In [ ]:
# Create a submission file for the test data

sample_submission = pd.DataFrame({'ID': test_data['ID'], 'Target': y_pred_test})
sample_submission.to_csv('sample_submission.csv', index=False)
print('Submission file created: sample_submission.csv')

Submission file created: sample_submission.csv
